In [135]:
import tensorflow as tf
import numpy as np
from os import listdir
from PIL import Image
import time
keras = tf.keras

In [2]:
Input = keras.layers.Input
BatchNormalization = keras.layers.BatchNormalization
Conv2D = keras.layers.Conv2D
UpSampling2D = keras.layers.UpSampling2D
Add = keras.layers.Add
Model = keras.models.Model
Dense = keras.layers.Dense
GlobalAveragePooling2D = keras.layers.GlobalAveragePooling2D
MaxPooling2D = keras.layers.MaxPool2D

In [3]:
input_layer = Input(shape=(224, 224, 3))

In [4]:
vgg16 = keras.applications.vgg16.VGG16(include_top=False,
                                       weights='imagenet',
                                       input_shape=(224, 224, 3),
                                       pooling=None, 
                                       input_tensor=input_layer)
layers = {layer.name: layer for layer in vgg16.layers}

In [5]:
block5_bn = BatchNormalization()(layers['block5_pool'].output)
block5_bn_conv = Conv2D(filters=512, kernel_size=1, activation='relu', padding='same')(block5_bn)
block5_up = UpSampling2D(size=(2, 2))(block5_bn_conv)

block4_bn = BatchNormalization()(layers['block4_pool'].output)
block45_add = Add()([block4_bn, block5_up])
block45_add_conv = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(block45_add)
block45_up = UpSampling2D(size=(2, 2))(block45_add_conv)

block3_bn = BatchNormalization()(layers['block3_pool'].output)
block34_add = Add()([block3_bn, block45_up])
block34_add_conv = Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(block34_add)
block34_up = UpSampling2D(size=(2, 2))(block34_add_conv)

block2_bn = BatchNormalization()(layers['block2_pool'].output)
block23_add = Add()([block2_bn, block34_up])
block23_add_conv = Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(block23_add)
block23_up = UpSampling2D(size=(2, 2))(block23_add_conv)

block1_bn = BatchNormalization()(layers['block1_pool'].output)
block12_add = Add()([block1_bn, block23_up])
block12_add_conv = Conv2D(filters=3, kernel_size=3, activation='relu', padding='same')(block12_add)

block12_up = UpSampling2D(size=(2, 2))(block12_add_conv)
output = Conv2D(filters=3, kernel_size=3, activation='sigmoid', padding='same')(block12_up)

g = Model(inputs=input_layer, outputs=output)
all_layers = {layer.name: layer for layer in g.layers}
for name, layer in layers.items():
    all_layers[name].trainable = False

In [6]:
block6_conv1 = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(layers['block5_pool'].output)
block6_conv2 = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(block6_conv1)
block6_conv3 = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(block6_conv2)
block6_pool = GlobalAveragePooling2D()(block6_conv3)
fc1 = Dense(512, activation='relu')(block6_pool)
output = Dense(1, activation='sigmoid')(block6_pool)

d = Model(inputs=input_layer, outputs=output)
all_layers = {layer.name: layer for layer in d.layers}
for name, layer in layers.items():
    all_layers[name].trainable = False

In [7]:
g.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [8]:
d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
generated_image = g(input_layer)
prediction = d(generated_image)
gan = Model(inputs=input_layer, outputs=[generated_image, prediction])

In [10]:
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
model_1 (Model)              (None, 224, 224, 3)       16533783  
_________________________________________________________________
model_2 (Model)              (None, 1)                 21794625  
Total params: 23,613,720
Trainable params: 8,896,088
Non-trainable params: 14,717,632
_________________________________________________________________


In [105]:
from os import listdir
batch_size = 200
steps_per_epoch = 60
epochs = 100

In [137]:
train_list = np.array(listdir('../data/train/'))
np.random.shuffle(train_list)
def image_generator(path_list, batch_size):
    while(True):
        random_indices = np.random.permutation(len(path_list))
        images = []
        for path in path_list[random_indices]:
            image = Image.open('../data/train/'+ path)
            image = np.array(image.convert('YCbCr'))
            images.append(image)
        yield images

In [138]:
train_image_gen = image_generator(train_list, 10)

In [141]:
start = time.time()
next(train_image_gen)
stop = time.time()

In [142]:
stop - start

465.4897918701172